In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split



In [ ]:
# 設定 Matplotlib 字體，確保 SHAP 可視化時能正常顯示中文
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
plt.rcParams['axes.unicode_minus'] = False

# 讀取 CSV 檔案
data_path = r"C:\thesis\code\Taipei_5x5\all_merged_5X5.csv"
df = pd.read_csv(data_path)

# 設定輸出目錄
result_dir = r"C:\thesis\code\result"
os.makedirs(result_dir, exist_ok=True)  # 若資料夾不存在則創建

# 處理角度變數
df['sin_max_gust_direction'] = np.sin(np.deg2rad(df['最大陣風風向']))  # 角度轉為sin值
df['cos_max_gust_direction'] = np.cos(np.deg2rad(df['最大陣風風向']))  # 角度轉為cos值
df['sin_wind_direction'] = np.sin(np.deg2rad(df['風向']))            # 角度轉為sin值
df['cos_wind_direction'] = np.cos(np.deg2rad(df['風向']))            # 角度轉為cos值

# 建立特徵名稱翻譯對應表（僅適用於決策樹）
feature_mapping = {
    '測站氣壓': 'Station_Pressure',
    '海平面氣壓': 'Sea_Level_Pressure',
    '氣溫': 'Temperature',
    '露點溫度': 'Dew_Point',
    '相對溼度': 'Relative_Humidity',
    '風速': 'Wind_Speed',
    '風向': 'Wind_Direction',
    '最大陣風': 'Max_Gust',
    '最大陣風風向': 'Max_Gust_Direction',
    '降水量': 'Precipitation',
    '降水時數': 'Precipitation_Hours',
    '日照時數': 'Sunshine_Hours',
    '全天空日射量': 'Global_Radiation',
    '能見度': 'Visibility',
    '紫外線指數': 'UV_Index',
    '總雲量': 'Total_Cloud_Cover',
    'hoilday': 'Holiday',
    'weekday': 'Weekday',
    '年': 'Year',
    '月': 'Month',
    '日': 'Day',
    '時': 'Hour'
}

# 保留原始 DataFrame
df_original = df.copy()

# 提取座標欄位
target_columns = [col for col in df.columns if '(' in col and ')' in col]
print("所有座標點：", target_columns)

# 替換 DataFrame 欄位名稱為英文（供 LightGBM 使用）
df_tree = df.rename(columns=feature_mapping)

# 設定 X (特徵) 與 y (目標)
X = df_original[list(feature_mapping.keys())]
y = df_original[target_columns]  # 這裡也可用 df[target_columns]，視需求而定

# 切分訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_tree = X_train.rename(columns=feature_mapping)
X_test_tree = X_test.rename(columns=feature_mapping)

# 定義類別型特徵
cat_features = ['Holiday', 'Month']

# 解析字串形式 "(經度, 緯度)" 的小函式
def parse_coord_string(coord_str):
    """
    將單個字串座標，如"(121.5, 25.0)"，解析成 (lon, lat) (float, float)
    """
    coord_str = coord_str.strip("() ")
    lon_str, lat_str = coord_str.split(",")
    return float(lon_str), float(lat_str)

# 初始化預測結果與 grid_ids (解決 grid_ids is not defined)
predictions = {}
grid_ids = []

# 定義遞迴提取決策樹特徵增益向量的函式
def get_feature_gain_vector(node, vector):
    if "split_feature" in node:
        feat_index = node["split_feature"]
        gain = node.get("split_gain", 0)
        vector[feat_index] += gain
    if "left_child" in node:
        get_feature_gain_vector(node["left_child"], vector)
    if "right_child" in node:
        get_feature_gain_vector(node["right_child"], vector)
    return vector

# 建立用於分群的其他列表
tree_vectors = []
geo_coords = []
# 建立用於儲存每個 target 的最佳決策樹根部規則
root_rules = {}
len(target_columns)

# 逐一為每個座標欄位 (target) 訓練模型

In [ ]:
for target in target_columns:
    print(f"訓練與預測 {target} 的決策樹...")

    # 建立 LightGBM Dataset
    train_data = lgb.Dataset(X_train_tree, label=y_train[target], categorical_feature=cat_features)
    test_data = lgb.Dataset(X_test_tree, label=y_test[target], reference=train_data, categorical_feature=cat_features)

    # 設定參數
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'seed': 42
    }
    evals_result = {}

    # 訓練模型
    lgb_model = lgb.train(
        params,
        train_data,
        num_boost_round=500,
        valid_sets=[test_data],
        valid_names=["valid_0"],
        callbacks=[lgb.early_stopping(stopping_rounds=25, verbose=True),
                   lgb.record_evaluation(evals_result),
                   lgb.log_evaluation(25)]
    )

    # 預測
    y_pred = lgb_model.predict(X_test_tree, num_iteration=lgb_model.best_iteration)
    predictions[target] = y_pred
    print(f"LightGBM 總樹數: {lgb_model.num_trees()}")

    # 繪製學習曲線
    if "valid_0" in evals_result and "rmse" in evals_result["valid_0"]:
        plt.figure(figsize=(8, 5))
        plt.plot(evals_result['valid_0']['rmse'], label="Validation RMSE", color="blue")
        plt.xlabel("Iterations")
        plt.ylabel("RMSE")
        plt.title(f"Learning Curve ({target})")
        plt.legend()
        learning_curve_path = os.path.join(result_dir, f"learning_curve_{target.replace(',', '_').replace(' ', '')}.png")
        plt.savefig(learning_curve_path, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"學習曲線已儲存至: {learning_curve_path}")
    else:
        print(f"無法繪製 {target} 的學習曲線，因為 evals_result 沒有數據。")

    # 找出最佳決策樹 (以總 split_gain 最大的那棵)
    model_dict = lgb_model.dump_model()
    tree_info = model_dict["tree_info"]
    def get_total_gain(node):
        gain = node.get("split_gain", 0)
        if "left_child" in node:
            gain += get_total_gain(node["left_child"])
        if "right_child" in node:
            gain += get_total_gain(node["right_child"])
        return gain

    tree_gains = []
    for tree in tree_info:
        total_gain = get_total_gain(tree["tree_structure"])
        tree_gains.append(total_gain)
    best_tree_index = np.argmax(tree_gains)


    plt.figure(figsize=(30, 18))
    lgb.plot_tree(lgb_model, tree_index=best_tree_index, show_info=['split_gain'])
    plt.title(f"Best Decision Tree for {target}")
    tree_plot_path = os.path.join(result_dir, f"best_tree_{target.replace(',', '_').replace(' ', '')}.png")
    plt.savefig(tree_plot_path, dpi=900, bbox_inches="tight")
    plt.close()
    print(f"最佳決策樹圖已儲存至: {tree_plot_path}")

    # SHAP 分析
    explainer = shap.TreeExplainer(lgb_model)
    shap_values = explainer.shap_values(X_test)
    shap_summary_path = os.path.join(result_dir, f"shap_summary_{target.replace(',', '_').replace(' ', '')}.png")
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values, X_test, show=False)
    plt.savefig(shap_summary_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"SHAP 特徵重要性圖已儲存至: {shap_summary_path}")

    shap_bar_path = os.path.join(result_dir, f"shap_bar_{target.replace(',', '_').replace(' ', '')}.png")
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
    plt.savefig(shap_bar_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"SHAP 特徵影響條形圖已儲存至: {shap_bar_path}")

    # 提取最佳決策樹的根部規則，並儲存到字典
    best_tree = tree_info[best_tree_index]["tree_structure"]
    root_rule = (best_tree.get("split_feature"), best_tree.get("threshold"))
    root_rules[target] = root_rule

    # 直接取該 target 欄位的代表經緯度（因為每個 target 就包含經緯度）
    geo_coords.append(target)
    grid_ids.append(target)

# 分群：利用決策樹根部相同規則的座標分在同一組

In [ ]:
# ------------------------
# 分群：利用決策樹根部使用相同特徵（忽略閾值）將座標分在同一組
# 先根據每個 target 的根部規則（存放在 root_rules 字典）進行分組
# 這裡只使用根部規則中的 split_feature（即 rule[0]）
features_order = list(X_train_tree.columns)
groups = {}
for target, rule in root_rules.items():
    feature_idx = rule[0]  # 僅使用 split_feature，不考慮 threshold
    groups.setdefault(feature_idx, []).append(target)

print("基於決策樹根部使用相同特徵分組的結果：")
for feature_idx, targets in groups.items():
    if feature_idx < len(features_order):
        feature_eng = features_order[feature_idx]
    else:
        feature_eng = str(feature_idx)
    feature_ch = reverse_mapping.get(feature_eng, feature_eng)
    print(f"規則 {feature_ch}：{targets}")

# 建立 target -> 分組標籤的對照表
group_labels = {}
for group_label, (feature_idx, targets) in enumerate(groups.items()):
    for t in targets:
        group_labels[t] = group_label

# 將分組結果輸出成 CSV（若想輸出成 Excel，可以使用 to_excel）
group_rows = []
for feature_idx, targets in groups.items():
    if feature_idx < len(features_order):
        feature_eng = features_order[feature_idx]
    else:
        feature_eng = str(feature_idx)
    feature_ch = reverse_mapping.get(feature_eng, feature_eng)
    rule_str = f"{feature_ch}"  # 只顯示特徵名稱
    group_rows.append({"規則": rule_str, "目標": ", ".join(targets)})

group_df = pd.DataFrame(group_rows)
excel_path = os.path.join(result_dir, "grouping_results.csv")
group_df.to_csv(excel_path, index=False, encoding='utf-8-sig')
print("分群結果已儲存至:", excel_path)

# 視覺化：將 geo_coords (存放 target 字串，格式 "(lon, lat)") 轉為數值型 tuple
# 注意：這裡 target_columns 與 geo_coords 順序一致
parsed_coords = [tuple(map(float, coord.strip("() ").split(","))) for coord in target_columns]

# 建立分組標籤列表，依據 grid_ids (grid_ids 存放 target)
group_label_list = [group_labels[t] for t in grid_ids]

# 取出各座標的經度與緯度
all_lons = [coord[0] for coord in parsed_coords]
all_lats = [coord[1] for coord in parsed_coords]

plt.figure(figsize=(10, 8))
plt.scatter(all_lons, all_lats, c=group_label_list, cmap='viridis', s=50, alpha=0.7)
plt.ticklabel_format(useOffset=False, style='plain', axis='both')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("基於決策樹根部使用相同特徵分群的地理分布")
plt.colorbar(label="Group Label")
grouping_path = os.path.join(result_dir, "geo_grouping_by_root_feature.png")
plt.savefig(grouping_path, dpi=300, bbox_inches="tight")
plt.close()
print("基於決策樹根部使用相同特徵的地理分群圖已儲存至:", grouping_path)


基於決策樹根部使用相同特徵分組的結果：
規則 時：['(121.505, 25.11)', '(121.505, 25.115)', '(121.51, 25.115)', '(121.51, 25.12)', '(121.515, 25.105)', '(121.515, 25.115)', '(121.52, 25.11)', '(121.52, 25.115)', '(121.525, 25.105)', '(121.525, 25.115)', '(121.525, 25.12)', '(121.535, 25.02)', '(121.535, 25.025)', '(121.535, 25.03)', '(121.535, 25.035)', '(121.54, 25.015)', '(121.54, 25.02)', '(121.54, 25.035)', '(121.545, 25.015)', '(121.545, 25.02)', '(121.545, 25.025)', '(121.545, 25.03)', '(121.545, 25.035)', '(121.55, 25.02)', '(121.55, 25.025)', '(121.55, 25.03)', '(121.55, 25.035)', '(121.555, 25.015)', '(121.555, 25.02)', '(121.555, 25.025)', '(121.555, 25.03)', '(121.555, 25.035)', '(121.575, 25.06)', '(121.575, 25.065)', '(121.575, 25.07)', '(121.575, 25.075)', '(121.575, 25.08)', '(121.58, 25.065)', '(121.58, 25.07)', '(121.58, 25.075)', '(121.58, 25.08)', '(121.585, 25.06)', '(121.585, 25.065)', '(121.585, 25.07)', '(121.59, 25.06)', '(121.59, 25.065)', '(121.59, 25.075)', '(121.59, 25.08)', '(121.5